In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [43]:
nan_df= pd.read_csv("clean_data/clean_dataframe_with_NaN.csv")
outcomes_a = pd.read_csv("data/outcomes/Outcomes-a.txt")
outcomes_b = pd.read_csv("data/outcomes/Outcomes-b.txt")
outcomes_c = pd.read_csv("data/outcomes/Outcomes-c.txt")

In [44]:
#merging with the outputs
nan_a = pd.merge(nan_df, outcomes_a, on="RecordID", how="inner")
nan_b = pd.merge(nan_df, outcomes_b, on="RecordID", how="inner")
nan_c = pd.merge(nan_df, outcomes_c, on="RecordID", how="inner")

In [45]:
nan_a.shape, nan_b.shape, nan_c.shape

((4000, 85), (4000, 85), (0, 85))

In [46]:
pd.options.display.max_columns = None
nan_a.head()

,RecordID,Age,Gender,ICUType,BMI,MechVent,MechVent_start,BUN_median,BUN_mad,BUN_slope,Creatinine_median,Creatinine_mad,FiO2_median,FiO2_max,FiO2_mad,GCS_median,GCS_mad,Glucose_48median,Glucose_median,Glucose_max,Glucose_mad,Glucose_slope,HCO3_median,HCO3_mad,HCT_median,HCT_mad,HR_median,HR_max,HR_mad,HR_slope,K_48median,K_median,K_max,K_mad,Lactate_median,Lactate_mad,Mg_median,Mg_max,Mg_mad,MAP_median,MAP_max,MAP_mad,MAP_last,MAP_slope,Na_median,Na_mad,Na_slope,NIMAP_median,NIMAP_max,NIMAP_mad,NIMAP_last,NIMAP_slope,PaCO2_median,PaCO2_mad,PaCO2_slope,PaO2_48median,PaO2_median,PaO2_max,PaO2_mad,PaO2_slope,pH_median,pH_mad,Platelets_median,Platelets_mad,Platelets_slope,Temp_48median,Temp_median,Temp_max,Temp_mad,Temp_last,Urine_median,Urine_max,Urine_mad,Urine_last,Urine_slope,WBC_median,WBC_mad,WBC_slope,Weight_24median,Weight_mad,SAPS-I,SOFA,Length_of_stay,Survival,In-hospital_death
0,132539.0,54.0,0.0,4.0,NaN,NaN,NaN,10.5,23.809524,-0.217391,0.75,6.666667,NaN,NaN,NaN,15.0,0.0,115.0,160.0,205.0,28.125000,-3.913043,27.0,3.703704,33.5,0.597015,72.0,86.0,11.111111,0.381291,4.0,4.20,4.4,4.761905,NaN,NaN,1.70,1.9,11.764706,NaN,NaN,NaN,NaN,NaN,136.5,0.366300,-0.043478,70.000,91.665,9.528571,79.33,0.075628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203.0,8.866995,-1.565217,37.90,37.80,38.2,0.793651,37.8,110.0,480.0,45.454545,280.0,3.449071,10.3,8.737864,-0.078261,NaN,NaN,6,1,5,-1,0
1,132540.0,76.0,1.0,2.0,26.228364,1.0,1.0,18.0,11.111111,0.125313,1.20,8.333333,0.45,0.75,11.111111,15.0,0.0,146.0,125.5,146.0,16.334661,1.518519,22.0,4.545455,28.8,2.430556,80.0,90.0,6.250000,-0.393039,3.5,3.90,4.3,10.256410,NaN,NaN,2.10,3.1,9.523810,79.0,98.0,5.063291,69.0,0.074967,137.0,1.459854,-0.148148,78.165,88.330,10.023668,68.33,-0.738433,40.0,15.000000,0.229198,111.0,153.0,445.0,42.483660,-3.926412,7.4,0.472973,181.0,9.392265,-1.296083,36.85,37.50,37.9,0.666667,37.1,95.0,600.0,50.000000,220.0,2.213950,13.1,1.526718,0.126301,80.6,0.0,16,8,8,-1,0
2,132541.0,44.0,0.0,3.0,NaN,1.0,10.0,3.0,0.000000,-0.113941,0.30,0.000000,0.50,0.50,0.000000,5.0,0.0,143.0,141.0,143.0,1.418440,0.174263,25.0,4.000000,28.7,1.045296,86.0,113.0,9.302326,-0.421952,3.4,3.35,3.7,5.970149,1.3,30.769231,1.80,1.9,5.555556,91.0,107.0,8.791209,92.0,-1.328571,138.0,0.724638,0.012064,96.670,110.000,6.548050,103.00,-0.210598,36.0,2.777778,-0.087892,173.0,120.5,232.0,44.813278,4.313901,7.5,0.066667,84.0,14.285714,1.089812,38.15,37.85,39.0,1.056803,37.2,100.0,425.0,50.000000,35.0,-1.012934,4.2,11.904762,0.058981,56.7,0.0,21,11,19,-1,0
3,132543.0,68.0,1.0,3.0,26.024291,NaN,NaN,20.0,15.000000,-0.408654,0.70,0.000000,NaN,NaN,NaN,15.0,0.0,117.0,117.0,129.0,9.401709,-0.182692,28.0,0.000000,36.8,1.902174,70.0,80.5,8.571429,0.053816,3.8,4.00,4.2,5.000000,NaN,NaN,2.10,2.1,0.000000,NaN,NaN,NaN,NaN,NaN,140.0,0.714286,-0.110577,82.835,91.670,4.225267,89.67,-0.064074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,315.0,9.841270,-2.870192,36.25,36.40,37.0,0.549451,37.0,650.0,700.0,7.692308,550.0,-1.254368,8.8,10.227273,-0.095192,84.6,0.0,7,1,9,575,0
4,132545.0,88.0,0.0,3.0,NaN,NaN,NaN,35.0,28.571429,-0.769231,1.00,0.000000,NaN,NaN,NaN,15.0,0.0,92.0,102.5,113.0,10.243902,-0.807692,19.0,5.263158,30.8,1.298701,72.0,94.0,5.555556,-0.370780,4.1,3.85,4.1,1.298701,NaN,NaN,1.55,1.6,3.225806,NaN,NaN,NaN,NaN,NaN,139.5,0.358423,-0.038462,75.000,105.700,6.886667,73.00,-0.292448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.0,5.825243,-0.461538,36.65,37.00,37.8,0.540541,36.7,50.0,150.0,20.000000,80.0,-0.264654,4.3,11.627907,0.038462,NaN,NaN,17,2,4,918,0


We will train the function with tha dataset a and then test it with b and c

STRATEGY TO WORK ON THE TRAINING SET (nan_a)
 - make a function to balance the df removing insights of the most frequent outcome priorizing those with higher number of nan
 - Keep the RecordID remaining in the nan_a to be used for training techniques

In [47]:
def balance_dataframe(df, target_column='In-hospital_death'):
    # Find the most frequent value in the target column
    most_frequent_value = df[target_column].mode()[0]
    
    # Subset the DataFrame with rows where the target column is the most frequent value
    most_frequent_df = df[df[target_column] == most_frequent_value]

    # Count NaN values in each row
    nan_counts = most_frequent_df.isnull().sum(axis=1)

    # Sort the DataFrame by the number of NaN values in ascending order
    sorted_df = most_frequent_df.iloc[nan_counts.argsort()]

    # Calculate the number of rows to drop to balance the DataFrame
    rows_to_drop = len(sorted_df) - (len(df) - len(sorted_df))

    # Drop rows with the most NaN values to balance the DataFrame
    balanced_df = df.drop(sorted_df.index[:rows_to_drop])

    return balanced_df


In [48]:
balan_nan_a = balance_dataframe(nan_a, target_column='In-hospital_death')


In [49]:
balan_nan_a['In-hospital_death'].value_counts()


In-hospital_death
0    554
1    554
Name: count, dtype: int64

In [50]:
outcomes_a_balanced = balan_nan_a[['RecordID', 'SAPS-I',	'SOFA',	'Length_of_stay',	'Survival',	'In-hospital_death']]

In [51]:
outcomes_a_balanced

,RecordID,SAPS-I,SOFA,Length_of_stay,Survival,In-hospital_death
0,132539.0,6,1,5,-1,0
4,132545.0,17,2,4,918,0
7,132551.0,19,8,6,5,1
8,132554.0,11,0,17,38,0
14,132573.0,12,3,8,740,0
...,...,...,...,...,...,...
3984,142640.0,11,7,6,-1,0
3985,142641.0,10,1,3,-1,0
3987,142649.0,12,2,6,1256,0
3996,142667.0,8,2,3,-1,0


In [52]:
outcomes_a_balanced.to_csv("data/outcomes/outcomes_a_balanced.csv", index = False)